In [1]:
!pip install workalendar
!pip install flask
!pip install flask_cors
!pip install kmodes
!pip install haversine

'DOSKEY'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


'DOSKEY'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


'DOSKEY'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


'DOSKEY'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


'DOSKEY'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


# 1. 가격 추천 파이프 라인

In [1]:
# 모듈 불러오기
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta, time
import json
import urllib
from urllib.request import Request, urlopen
from workalendar.asia import SouthKorea


In [2]:
# 날씨 가져오는 함수
def Check_weather(time, address):
    # 도시, 시구까지 단위로 쪼개기
    location = ' '.join(address.split()[:2])
    # url 생성
    url = 'https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=1&ie=utf8&query=' + location + '날씨'
    # 날씨 추출
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')

    # 오류 체크
    ErrorCheck = soup.find('ul', {'class': 'week_list'})

    if 'None' in str(ErrorCheck):
        return "Error! 지역 검색 오류!"
    else:
        # 오늘날씨 받아오기
        today_date = date.today()
        # 주간예보
        weekly_weather = soup.find('ul', {'class': 'week_list'})
        # 출발일시 전처리
        datetime_all = datetime.strptime(time, '%Y.%m.%d %H:%M')
        start_date = datetime_all.date()
        # 오늘 날짜와 비교하여 가져올 날짜 확인
        date_diff = (start_date - date.today()).days + 1
        # 날씨
        day_all = weekly_weather.select_one(f'li:nth-child({date_diff})')
        day_date = day_all.find('span', {'class': 'date'}).text
        day_date = datetime.strptime('2024.'+day_date.rstrip("."), '%Y.%m.%d').date()
        day_morning = day_all.select_one('div.cell_weather > span:nth-child(1) > i > span').text
        day_evening = day_all.select_one('div.cell_weather > span:nth-child(2) > i > span').text

        # 오전과 오후를 나눠 실시
        if datetime_all.hour < 12:
                if '비' in day_morning or '눈' in day_morning:
                    return 1
                else:
                    return 0
        else:
            if '비' in day_evening or '눈' in day_evening:
                return 1
            else:
                return 0

# 심야시간 할증 가져오는 함수
def calculate_overnight_percentage(start_time_str, end_time_str):
    # 시작 시간과 종료 시간을 datetime 객체로 변환
    start_time = datetime.strptime(start_time_str, '%Y.%m.%d %H:%M')
    end_time = datetime.strptime(end_time_str, '%Y.%m.%d %H:%M')
    total_duration = end_time - start_time

    # 상차시간을 한시간 더하여 새로운 시작 시간 생성
    # 상차시간도 심야시간에 포함하기로 함
    start_time = datetime.strptime(start_time_str, '%Y.%m.%d %H:%M') # + timedelta(hours=1)

    # 심야시간 범위를 datetime 객체로 생성
    # 시작시간이 6시이전이라면 심야시작이 당일 6시, 끝시간은 당일 22시
    if start_time.time() < datetime.strptime('06:00', '%H:%M').time():
        overnight_start_time = start_time.replace(hour=6, minute=0, second=0)
        overnight_end_time = start_time.replace(hour=22, minute=0, second=0)

        # 5
        if start_time < overnight_start_time and end_time <= overnight_end_time:
            overnight_duration = overnight_start_time - start_time
        # 6
        elif start_time < overnight_start_time and end_time > overnight_end_time:
            overnight_duration = (overnight_start_time - start_time) + (end_time - overnight_end_time)

        percentage = (overnight_duration / total_duration) * 100

    # 시작시간이 6시 이후인 경우, 심야시작이 당일 22시, 끝시간은 익일 6시
    elif start_time.time() >= datetime.strptime('06:00', '%H:%M').time():
        overnight_start_time = start_time.replace(hour=22, minute=0, second=0)
        overnight_end_time = start_time.replace(hour=6, minute=0, second=0)  + timedelta(days=1)

        # 1
        if start_time <= overnight_start_time and end_time <= overnight_end_time:
            overnight_duration = end_time - overnight_start_time
        # 2,3,4
        elif start_time > overnight_start_time and end_time <= overnight_end_time:
            overnight_duration = min(end_time, overnight_end_time) - start_time
        # 할증시간대를 모두 운전하고 앞뒤로도 더 운전하는 경우
        else:
            overnight_duration = min(end_time, overnight_end_time) - start_time

        percentage = (overnight_duration / total_duration) * 100

    # 그외 시간안에 들어오거나 조금 넘어도 퍼센트가 넘지않음
    else:
        percentage = 0

    if percentage >= 50:
        return '할증'
    else:
        return 'NO할증'


# 위도 경도를 가져오는 함수
def get_location(loc) :
    url = f"https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode?query=" \
    			+ urllib.parse.quote(loc)

    # 주소 변환
    request = urllib.request.Request(url)
    request.add_header('X-NCP-APIGW-API-KEY-ID', client_id)
    request.add_header('X-NCP-APIGW-API-KEY', client_secret)

    response = urlopen(request)
    res = response.getcode()

    if (res == 200) : # 응답이 정상적으로 완료되면 200을 return한다
        response_body = response.read().decode('utf-8')
        response_body = json.loads(response_body)
        # print(response_body)
        # 주소가 존재할 경우 total count == 1이 반환됨.
        if response_body['meta']['totalCount'] == 1 :
        	# 위도, 경도 좌표를 받아와서 return해 줌.
            lat = response_body['addresses'][0]['y']
            lon = response_body['addresses'][0]['x']
            return (lon, lat)
        else :
            print('location not exist')

    else :
        print('ERROR')


# 거리, 통행비, 유류비 받아오는 함수
option = 'traoptimal' # 실시간 최적
fueltype = 'diesel' # 유류비 계산시 사용
cartype = '1'
mileage = 9.6
def get_distance_toll_fuel(start, arrive, start_time, cartype = cartype, mileage = mileage, fueltype = fueltype) :

    # 주어진 문자열을 datetime 객체로 변환
    start_time = datetime.strptime(start_time, '%Y.%m.%d %H:%M')
    # 1시간을 더하기 = 상차시간 더하기
    new_time = start_time + timedelta(hours=1)
    # 새로운 시간을 문자열로 변환
    new_time_str = new_time.strftime('%Y.%m.%d %H:%M:%S')

    # start=/goal=/(waypoint=)/(option=) 순으로 request parameter 지정
    url = "https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving"
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }
    # 요청할 파라미터 설정
    # start와 goal은 예시 좌표이며, 실제 애플리케이션에서는 사용자가 지정한 위치 좌표를 사용해야 합니다.
    params = {
        "start": start[0] + ',' + start[1],
        "goal": arrive[0] + ',' + arrive[1],
        "option": "traoptimal",
        "mileage": mileage,  # 연비
        "cartype": cartype,  # cartype 변경 (1: 승용차, 2: 화물차 등)
        "fueltype": fueltype,
        "startTime": new_time_str
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200 :
        data_ = response.json()

        result = {
            'total_distance' : data_['route']['traoptimal'][0]['summary']['distance'],
            'total_duration' : data_['route']['traoptimal'][0]['summary']['duration'],
            'fares' : {'toll' : data_['route']['traoptimal'][0]['summary']['tollFare'],
                    'fuel' : data_['route']['traoptimal'][0]['summary']['fuelPrice']}
        }

        total_distance = result['total_distance']/1000
        fares_toll = result['fares']['toll']
        fares_fuel = result['fares']['fuel']

        def convert_time(milliseconds):
            seconds = milliseconds / 1000.0
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            return hours, minutes
        hours, minutes = convert_time(result['total_duration'])
        total_duration = time(hours, minutes)

        return total_distance, total_duration, fares_toll, fares_fuel
    else :
        print('ERROR')


# 일요일 및 공휴일정보 받아오는 함수
def is_holiday(start_date_str):
    start_date = datetime.strptime(start_date_str, '%Y.%m.%d %H:%M').date()
    weekday = start_date.weekday()

    if weekday == 6:
        return "공휴일"
    else:
        cal = SouthKorea()
        holidays = cal.holidays(2024)

        for holiday in holidays:
            if start_date in holiday:
                return "공휴일"
            else:
                return "NO공휴일"


# 대기시간 반환 함수
# 대기시간 = (도착시간-출발시간) - (차량운행시간 + 30분)
def calculate_waiting_time(start_time_str, end_time_str, drive_time):
    # 타입 변환
    start_date = datetime.strptime(start_time_str, '%Y.%m.%d %H:%M')
    end_date = datetime.strptime(end_time_str, '%Y.%m.%d %H:%M')

    # (도착시간-출발시간)
    all_time = (end_date - start_date)

    # 차량운영시간 먼저 제거
    all_minus_pickup_time = all_time - timedelta(minutes=30)

    # 대기시간
    # drive_time = 네이버 maps api에서 받아온 시간
    waiting_time = all_minus_pickup_time - timedelta(hours=drive_time.hour, minutes=drive_time.minute)
    total_hours = waiting_time.days * 24 + waiting_time.seconds / 3600
    # print(total_hours)

    # 두시간 초과 여부 확인
    if total_hours > 2:
        # 초과하는 경우 30분단위로 나눠 몇개인지 출력
        cnt_30_minutes = int(((total_hours - 2) / 0.5)) + 1
        return cnt_30_minutes
    else:
        return 0

# 감춰야할 정보
client_id = 'z94i3c62lh'
client_secret = 'AEwGtEnTO70eCAgGeEjaTrhSt99YVS68eeEv06J3'

# 톤수별로 다른 파라미터들 가져오는 함수
def get_ton_data(ton, driving_option):
    ton_info = {
        1: {'toll': 1, 'mileage': 9.6, 'car_option_fee': 10000, 'option_fee': {1: 0, 2: 20000, 3: 40000}},
        1.4: {'toll': 1, 'mileage': 8.5, 'car_option_fee': 10000, 'option_fee': {1: 0, 2: 20000, 3: 40000}},
        2.5: {'toll': 2, 'mileage': 8.5, 'car_option_fee': 20000, 'option_fee': {1: 0, 2: 10000, 3: 30000}},
        5: {'toll': 2, 'mileage': 5.4, 'car_option_fee': 20000, 'option_fee': {1: 0, 2: 10000, 3: 30000}},
        8: {'toll': 3, 'mileage': 4.2, 'car_option_fee': 30000, 'option_fee': {1: 0, 2: 10000, 3: 20000}},
        11: {'toll': 4, 'mileage': 4.2, 'car_option_fee': 30000, 'option_fee': {1: 0, 2: 10000, 3: 20000}},
        25: {'toll': 5, 'mileage': 3.6, 'car_option_fee': 30000, 'option_fee': {1: 0, 2: 10000, 3: 20000}},
    }

    ton_data = ton_info.get(ton)
    if ton_data is None:
        return None  # 지원하지 않는 톤수인 경우 None 반환

    toll = ton_data['toll']
    mileage = ton_data['mileage']
    car_option_fee = ton_data['car_option_fee']

    driving_option_fee = ton_data['option_fee'].get(driving_option, 0)

    return toll, mileage, car_option_fee, driving_option_fee

import os
import pickle

# 모델 파일들이 저장된 폴더 경로
folder_path = "./ton/"

# 불러온 모델들을 저장할 딕셔너리
models = {}

# 7개의 모델 파일 불러오기
for i in [1, 1.4, 2.5, 5, 8, 11, 25]:
    file_name = f"{i}ton.pkl"
    file_path = os.path.join(folder_path, file_name)

    # 파일이 존재하는 경우에만 불러오기
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            models[f"{i}ton_model"] = pickle.load(file)
    else:
        print(f"파일 {file_name}이 존재하지 않습니다.")

# 불러온 모델들 확인
print(models)

# 냉장냉동무진동
def get_freeze_option_fee(distance_fee, fridge, no_vibration_fee):
    # 냉장
    fridge_fee = distance_fee * 0.3 if fridge == "yes" else 0
    # 무진동
    no_vibration_fee = distance_fee * 0.3 if no_vibration_fee == "yes" else 0

    freeze_option_fee = fridge_fee + no_vibration_fee

    return freeze_option_fee

# 상하차 옵션(상하차, 상하차+운반)
# driving_option_fee

# 운송옵션(탑윙)

def get_car_fee(distance_fee, car_option, fridge, no_vibration_fee):
    if car_option == 1:
        car_fee = 0
    elif fridge == "yes" or no_vibration_fee == "yes":
        car_fee = 0
    else:
        car_fee = car_option

    return car_fee

def cal_model(data):
    start_time, arrive_time, start_add, arrive_add, fridge, no_vibration, driving_option, car_option, ton, one_way = data
    
    # 톤수별 정보 가져오기
    toll, mileage, car_option_fee, driving_option_fee = get_ton_data(ton, driving_option)

    # 위도 경도 가져오기
    start_add_longitude_latitude = get_location(start_add)
    arrive_add_longitude_latitude = get_location(arrive_add)


    # 거리계산이 시작!

    # 거리, 시간, 통행료, 유류비
    drive_distance, drive_time, toll_fee, fuel_fee = get_distance_toll_fuel(start_add_longitude_latitude, arrive_add_longitude_latitude, start_time, cartype = toll, mileage = mileage, fueltype = fueltype)
    pred_data = [[drive_distance, float(start_add_longitude_latitude[0]), float(start_add_longitude_latitude[1]), float(arrive_add_longitude_latitude[0]), float(arrive_add_longitude_latitude[1])]]

    # 거리에 맞춘 비용
    # ton_str = str(ton)
    distance_fee = models[f"{str(ton)}ton_model"].predict(pred_data)[0]

    # 공휴일 계산
    if is_holiday(start_time) == "공휴일":
        holiday_fee = distance_fee * 0.2
    else:
        holiday_fee = 0

    # 심야시간
    if calculate_overnight_percentage(start_time, arrive_time) == "할증":
       overnight_fee = distance_fee * 0.2
    else:
        overnight_fee = 0


    # 대기시간
    waiting_time_fee = 20000 * calculate_waiting_time(start_time, arrive_time, drive_time)

    # 날씨 직전의 가격
    final_fee = distance_fee + holiday_fee + overnight_fee + get_freeze_option_fee(distance_fee, fridge, no_vibration) + driving_option_fee + get_car_fee(distance_fee, car_option, fridge, no_vibration) + waiting_time_fee + toll_fee + fuel_fee

    #날씨 추가

    weather_cnt = Check_weather(start_time, start_add) + Check_weather(arrive_time, arrive_add)
    if weather_cnt == 0:
        weather_fee = 0
    elif weather_cnt == 1:
        weather_fee = final_fee * 0.1
    elif weather_cnt == 2:
        weather_fee = final_fee * 0.2

    real_final_fee = final_fee + weather_fee


    # 왕복?
    if one_way == "no":
        real_final_fee = real_final_fee * 1.8
    location = start_add.split()[0]

    minus_location = ['경기도', '경상남도', '부산광역시']
    still_location = ['전라남도', '강원도', '인천광역시', '충청남도', '전라북도', '충청북도', '서울특별시']
    plus_location = ['경상북도', '광주광역시', '대전광역시', '대구광역시', '울산광역시', '세종특별자치시']

    if location in minus_location:
        location_fee = -(real_final_fee * 0.05)
    elif location in still_location:
        location_fee = 0
    elif location in plus_location:
        location_fee = real_final_fee * 0.05
    
    return real_final_fee


{'1ton_model': LGBMRegressor(), '1.4ton_model': LGBMRegressor(), '2.5ton_model': LGBMRegressor(), '5ton_model': LGBMRegressor(), '8ton_model': LGBMRegressor(), '11ton_model': LGBMRegressor(), '25ton_model': LGBMRegressor()}


# 2. 추천 알고리즘 파이프라인

In [3]:
# 모듈 불러오기
import pandas as pd
import numpy as np
from kmodes.kprototypes import KPrototypes
from haversine import haversine
import pickle

In [4]:
# 데이터 전처리 및 모델링 코드
# user_data:차주 데이터
def model_pre():
    drop_cols = ['car_type','num','Departure', 'Destination']

    with open('kproto_model.pkl', 'rb') as f:
        kproto = pickle.load(f)
    with open('clusters_model.pkl', 'rb') as f:
        clusters = pickle.load(f)


    # 군집리스트를 만들어서 나중에 화주 넣기
    customer_clusters = [[] for _ in range(len(set(kproto.labels_)))]

    return kproto,clusters, customer_clusters

# 새로운 화주 데이터를 K-Prototypes 모델에 넣어 해당 데이터가 어떤 군집에 속하는지 알아냅니다.
def comp_cluster(new_customers, customer_clusters, kproto):

    for idx, new_customer in new_customers.iterrows():
        # 새로운 화주 데이터를 K-Prototypes 모델에 넣어 해당 데이터가 어떤 군집에 속하는지 알아냅니다.
        customer = pd.DataFrame([new_customer])
        customer.drop(['num','Departure','Destination'],axis=1, inplace=True)

        # new_customer는 Series 형태이므로 DataFrame으로 변환해줍니다.
        cluster_label = kproto.predict(customer, categorical=[0,2,3,4]) #1

        # 해당 군집에 새롭게 화주를 추가합니다.
        customer_clusters[cluster_label[0]].append(new_customer['num'])

    return customer_clusters


# 추천할 차주 데이터 넣기
def clusterfilter(new_driver_features, current_lat_col, current_lon_col, new_customers, kproto, customer_clusters):
    new_driver = new_driver_features

    driver = new_driver.drop(['num','Departure','Destination','distance'],axis=1)
    cluster_label = kproto.predict(driver, categorical=[0,2,3,4])

    df = pd.DataFrame()
    for i in customer_clusters[cluster_label[0]]:
        temp_df = new_customers[new_customers['num'] == i]
        df = pd.concat([df, temp_df], ignore_index=True)
    

    df_f = df.loc[(df['is_freezer']==new_driver['is_freezer'].values[0]) & (df['T']==new_driver['T'].values[0]) & (df['vibration_free']==new_driver['vibration_free'].values[0])]
    
    # 위에서 정의한 함수를 사용하여 haversine 거리 계산
    df_f['Haversine_Distance'] = calculate_haversine_distance(df_f, 'Departure_latitude', 'Departure_longitude', current_lat_col, current_lon_col)
    df_f['differ_Distance'] = calculate_differ_distance(df_f, 'Departure_latitude', 'Departure_longitude', 'Destination_latitude', 'Destination_longitude')
    # 장거리 단거리 여부
    if new_driver_features['distance'].item()==1:
        df_f = df_f[df_f['differ_Distance']>=110]
    else:
        df_f = df_f[df_f['differ_Distance']<110]

    df_f = df_f.sort_values('Haversine_Distance')
    return df_f[:3]


# 출발지와 현재 거리 계산 함수
def calculate_haversine_distance(df, start_lat_col, start_lon_col, current_lat_col, current_lon_col):
    # Apply haversine function to each row
    distances = df.apply(lambda row: haversine((row[start_lat_col], row[start_lon_col]),
                                               (current_lat_col, current_lon_col)), axis=1)
    return distances


# 현재와 도착지의 거리 계산 함수
def calculate_differ_distance(df, start_lat_col, start_lon_col, current_lat_col, current_lon_col):

    distances = df.apply(lambda row: haversine((row[start_lat_col], row[start_lon_col]),
                                               (row[current_lat_col], row[current_lon_col])), axis=1)
    return distances





# 3. 연동

In [5]:
# 모듈 불러오기
import pandas as pd
import numpy as np
from kmodes.kprototypes import KPrototypes
from haversine import haversine
import pickle

In [ ]:

from flask import Flask, request, jsonify 
from flask_cors import CORS
import urllib

app = Flask(__name__)
CORS(app)
@app.route('/predict1', methods=['GET', 'POST'])
def get_prediction():
    data = request.json  # 입력 데이터를 받습니다.
    
    start_time = data['arrivalDateTime']
    end_time = data['departureDateTime']
    start_address = data['address']['address']
    end_address = data['location']['roadAddress']
    fridge = data['isChecked2']
    no_vibration = data['isChecked1']
    driving_option = data['selectedButton'] # 상하차
    car_type = data['selectedBox']
    ton = data['tonnage']
    oneway = data['selected2']

    #시간 전처리
    start_time = start_time.replace('-', '.').replace('T', ' ')
    end_time = end_time.replace('-', '.').replace('T', ' ')
    
    # 냉동냉장 전처리
    if fridge == True:
        fridge = "yes"
    else:
        fridge = "no"
        
    # 무진동
    if no_vibration == True:
        no_vibration = "yes"
    else:
        no_vibration = "no"
        
    #상하차
    if driving_option == '본인이 직접 옮김':
        driving_option = 1
    elif driving_option == '상하차만 도움':
        driving_option = 2
    else:
        driving_option = 3
    
    # 차 종류
    if car_type == '카고':
        car_type = 1
    elif car_type == '윙바디':
        car_type = 2
    else:
        car_type = 3
        
    # ton
    ton = float(ton[:-1])
    if '0' in str(ton):
        ton = int(ton)
    
    #편도
    if oneway == '편도':
        oneway = 'yes'
    else:
        oneway = 'no'
    tmp_data = [start_time, end_time, start_address, end_address, fridge, no_vibration, driving_option, car_type, ton, oneway]
    result = cal_model(tmp_data)
    result = format(int(round(result,-1)), ',')
    return {'result': result}

@app.route('/predict2', methods=['GET', 'POST'])
def recommand():
    data = request.json  # 입력 데이터를 받습니다.
    print(data)
    regist = data['registInfoList']
    location = data['location']
    car_data = data['carrierInfoList']
    
    regist_df = pd.DataFrame(regist)
    # 각 출발지 도착지에 대한 위도경도, 업종에 대한 정보 X
    regist_df = regist_df[['id', 'sectors', 'tonnage', 'isChecked1', 'isChecked2', 'arrival_Address', 'departure_address', 'selected2']]
    regist_df.rename(columns = {'arrival_Address' : 'area1', 'departure_address' : 'area2'}, inplace = True)
    current_lat_col = location['coordinates']['lat']
    current_lon_col = location['coordinates']['lng']
    
    # 각 선호지에 대한 위도 경도 여기에는 왜 왕복여부가 들어가는거지
    car_df = pd.DataFrame(car_data)
    car_df = car_df[['id', 'sectors', 'tonnage', 'isChecked1', 'isChecked2', 'area1', 'area2', 'distance']]
    car_df['selected2'] = 0
    
    
    for tmp in [regist_df, car_df]:
        for index, row in tmp.iterrows():
            tmp['area1_long'], tmp['area1_lat'] = get_location(row['area1'])
            tmp['area2_long'], tmp['area2_lat'] = get_location(row['area2'])


        # 카테고리
        dic = {'농림축산업':1, '광업':2, '제조업':3,'건설업':4,'도소매업':5, '숙박 및 서비스업':6, '기타 서비스업':7, '공공기관':8}
        tmp['sectors'] = tmp['sectors'].map(dic)

        # ton
        tmp_ton = list()
        for ton in tmp['tonnage']:
            ton = float(ton[:-1])
            if '0' in str(ton):
                ton = int(ton)
            tmp_ton.append(ton)
        tmp['tonnage'] = tmp_ton

        # 냉동냉장 전처리
        tmp['isChecked1'] = tmp['isChecked1'].map({True: 1, False: 0})

        # 무진동
        tmp['isChecked2'] = tmp['isChecked2'].map({True: 1, False: 0})

        #왕복
        tmp['selected2'] = tmp['selected2'].map({'왕복': 1, '편도': 0})
        if 'distance' in tmp.columns:
            tmp['distance'] = tmp['distance'].map({'장거리': 1, '단거리': 0})
        tmp.rename(columns = {'id' : 'num',	'sectors' : 'category',	'tonnage' : 'T', 'isChecked1' : 'is_freezer', 'isChecked2' : 'vibration_free',
                              'area1' : 'Departure', 'area2' : 'Destination',	'selected2' : 'transport_type',	'area1_long' : 'Destination_longitude',
                              'area1_lat' : 'Destination_latitude',	'area2_long' : 'Departure_longitude',	'area2_lat' : 'Departure_latitude'}, inplace = True)
        tmp = tmp
    regist_df = regist_df.astype({'Departure_longitude' : 'float', 'Departure_latitude' : 'float', 'Destination_latitude' : 'float', 'Destination_longitude' : 'float'})
    car_df = car_df.astype({'Departure_longitude' : 'float', 'Departure_latitude' : 'float', 'Destination_latitude' : 'float', 'Destination_longitude' : 'float'})
    car_df['transport_type'] = 0
    
    kproto,clusters, customer_clusters = model_pre()
    
    customer_clusters = comp_cluster(regist_df, customer_clusters, kproto)
    res = clusterfilter(car_df, current_lat_col, current_lon_col, regist_df, kproto, customer_clusters)
    display(res)
    return {'result': res['num'].to_dict()}
    
    
if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Jan/2024 09:13:20] "OPTIONS /predict2 HTTP/1.1" 200 -
[2024-01-10 09:13:20,961] ERROR in app: Exception on /predict2 [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  

{'registInfoList': [{'id': 1, 'username': 'a', 'selected2': '편도', 'arrivalDateTime': '2024-01-09T16:32', 'departureDateTime': '2024-01-09T04:32', 'tonnage': '2.5톤', 'selectedBox': '카고', 'isChecked1': False, 'isChecked2': True, 'text': '', 'selectedSize': '소형', 'selectedBoxNew': '박스', 'weight': 30, 'textAreaValue': '', 'selectedValue': '15', 'selectedButton': '상하차만 도움', 'headquarters2': 'zz', 'headquarters3': 'dd', 'arrival_Code': '34694', 'arrival_Address': '대전 동구 대전로 387 (가오동)', 'arrival_detailAddress': '', 'departure_code': '02592', 'departure_address': '서울 동대문구 서울시립대로 5 (답십리동, 신답극동아파트)', 'departure_detailAddress': '', 'currentDateTime': '2024-01-09T16:32:56', 'distance': 169.06, 'durationHour': 2, 'durationMin': 39, 'yourcost': 500000, 'status': 0, 'sectors': '광업', 'shipperInfo': {'shipMember': 1, 'name': 'a', 'phone': 'aa', 'password': '$2a$10$kjgxCow.tHVqEEI1NJHxz.2fjIe7QwUJNnWNDYgjLSE4K5r5T78ta', 'account': 'a', 'email': 'aa@aa.aa', 'cname': 'a'}, 'carrierInfo': None}, {'id': 2, 

127.0.0.1 - - [10/Jan/2024 09:13:31] "OPTIONS /predict2 HTTP/1.1" 200 -
[2024-01-10 09:13:31,690] ERROR in app: Exception on /predict2 [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app

{'registInfoList': [], 'location': {'loaded': False, 'coordinates': {'lat': '', 'lng': ''}}, 'carrierInfoList': []}


...

{'registInfoList': [], 'location': {'loaded': False, 'coordinates': {'lat': '', 'lng': ''}}, 'carrierInfoList': []}


127.0.0.1 - - [10/Jan/2024 09:51:59] "OPTIONS /predict2 HTTP/1.1" 200 -
[2024-01-10 09:51:59,075] ERROR in app: Exception on /predict2 [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app

{'registInfoList': [], 'location': {'loaded': False, 'coordinates': {'lat': '', 'lng': ''}}, 'carrierInfoList': []}


127.0.0.1 - - [10/Jan/2024 09:56:17] "OPTIONS /predict2 HTTP/1.1" 200 -
[2024-01-10 09:56:17,720] ERROR in app: Exception on /predict2 [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask_cors\extension.py", line 176, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
                                                ^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\flask\app

{'registInfoList': [], 'location': {'loaded': False, 'coordinates': {'lat': '', 'lng': ''}}, 'carrierInfoList': []}
